In [74]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import os

### 0) Configuração de pré-pesquisa
Aqui será configurado as informaçoes fornecidas para a pesquisa, como a data de inicio e fim da busca, tamanho da visualização e UF(Unidade Federativa) que será pesquisado os discursos.

In [75]:
#formato de data = {dd, mm, aaaa}
data_inicio = [28, 3, 2022]
data_fim = [28, 4, 2022]
#tamanho = quantidade de linhas de informações por página (limite 1788 pq? não sabemos)
tamanho = 100
current_page = 1
#Estado onde será realizada a pesquisa ('' para todos)
UF = ''

url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txIndexacao=&BasePesq=plenario&txOrador=&txPartido=&dtInicio={data_inicio[0]}/{data_inicio[1]}/{data_inicio[2]}&dtFim={data_fim[0]}/{data_fim[1]}/{data_fim[2]}&txUF={UF}&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=ASC&PageSize={tamanho}&txTexto=&txSumario='
      

In [76]:
html = requests.get(url)

content = html.content

soup = BeautifulSoup(content, 'html.parser')

In [77]:
quantidade_de_discursos= []
qpt = 0 #quantidade de paginas totais

for links in soup.find_all('span', attrs={'class':"visualStrong"}):
    quantidade_de_discursos.append(links.contents)
qpt= int(((int((quantidade_de_discursos[len(quantidade_de_discursos)-1][0]).replace('.','')))/tamanho)+1)
qpt

22

In [78]:

def UrlToSoup(url):
    html = requests.get(url)
    content = html.content
    return BeautifulSoup(content, 'html.parser')

### 1) Criar um data frame com as informações do site

In [79]:
url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF={UF}&dtInicio={data_inicio[0]}%2F{data_inicio[1]}%2F{data_inicio[2]}&dtFim={data_fim[0]}%2F{data_fim[1]}%2F{data_fim[2]}&txSessao=&listaTipoFala=&listaFaseSessao=&listaTipoInterv=&txAparteante=&txTexto=&txSumario=&txIndexacao=&BasePesq=plenario&CampoOrdenacao=dtSessao&PageSize={tamanho}&TipoOrdenacao=ASC&btnPesqAvan=Pesquisar'

html = requests.get(url)

content = html.content

soup = BeautifulSoup(content, 'html.parser')

print(url)

https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF=&dtInicio=28%2F3%2F2022&dtFim=28%2F4%2F2022&txSessao=&listaTipoFala=&listaFaseSessao=&listaTipoInterv=&txAparteante=&txTexto=&txSumario=&txIndexacao=&BasePesq=plenario&CampoOrdenacao=dtSessao&PageSize=100&TipoOrdenacao=ASC&btnPesqAvan=Pesquisar


In [134]:
data_frame = [None]*qpt
for i in range(qpt):
    current_page = i+1
    url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txIndexacao=&BasePesq=plenario&txOrador=&txPartido=&dtInicio={data_inicio[0]}/{data_inicio[1]}/{data_inicio[2]}&dtFim={data_fim[0]}/{data_fim[1]}/{data_fim[2]}&txUF={UF}&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=ASC&PageSize={tamanho}&txTexto=&txSumario='
    data_frame[i] = pd.read_html(url)


In [104]:
data_frame[0][0]

,Data,Sessão,Fase,Discurso,Sumário,Orador,Hora,Publicação
0,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022
1,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022
2,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022
3,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022
4,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022
...,...,...,...,...,...,...,...,...
95,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"REINHOLD STEPHANES JUNIOR, PSD-PR",15h52,DCD 30/03/2022
96,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"FRANCO CARTAFINA, PP-MG",15h56,DCD 30/03/2022
97,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"MARCEL VAN HATTEM, NOVO-RS",16h00,DCD 30/03/2022
98,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"DANIEL SILVEIRA, UNIÃO-RJ",16h04,DCD 30/03/2022


### 2) Criar uma lista com todos os links para os discursos

In [82]:
link_infos= []
soup = []
temp = []
for i in range(qpt):
        current_page = i+1
        url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txIndexacao=&BasePesq=plenario&txOrador=&txPartido=&dtInicio={data_inicio[0]}/{data_inicio[1]}/{data_inicio[2]}&dtFim={data_fim[0]}/{data_fim[1]}/{data_fim[2]}&txUF={UF}&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=ASC&PageSize={tamanho}&txTexto=&txSumario='
        soup.append(UrlToSoup(url))
        for links in soup[i].find_all('a', attrs={'title':"Íntegra do Discurso"}):
            temp.append((links['href']))
        link_infos.append(temp)
        temp = []


In [83]:
link_infos

[['TextoHTML.asp?etapa=5\r\n\t\t\t\t\t\t\t&nuSessao=013.4.56.N\r\n\t\t\t\t\t\t\t&nuQuarto=0\r\n\t\t\t\t\t\t\t&nuOrador=0\r\n\t\t\t\t\t\t\t&nuInsercao=3\r\n\t\t\t\t\t\t\t&dtHorarioQuarto=12:37\r\n\t\t\t\t\t\t\t&sgFaseSessao=OD\r\n\t\t\t\t\t\t\t&Data=28/04/2022\r\n\t\t\t\t\t\t\t&txApelido=AFONSO FLORENCE, PT-BA\r\n\t\t\t\t\t\t\t&txFaseSessao=Ordem do Dia\r\n\t\t\t\t\t\t\t&txTipoSessao=Extraordinária - CN\r\n\t\t\t\t\t\t\t&dtHoraQuarto=12:37\r\n\t\t\t\t\t\t\t&txEtapa=',
  'TextoHTML.asp?etapa=5\r\n\t\t\t\t\t\t\t&nuSessao=013.4.56.N\r\n\t\t\t\t\t\t\t&nuQuarto=0\r\n\t\t\t\t\t\t\t&nuOrador=0\r\n\t\t\t\t\t\t\t&nuInsercao=23\r\n\t\t\t\t\t\t\t&dtHorarioQuarto=12:37\r\n\t\t\t\t\t\t\t&sgFaseSessao=OD\r\n\t\t\t\t\t\t\t&Data=28/04/2022\r\n\t\t\t\t\t\t\t&txApelido=AFONSO FLORENCE, PT-BA\r\n\t\t\t\t\t\t\t&txFaseSessao=Ordem do Dia\r\n\t\t\t\t\t\t\t&txTipoSessao=Extraordinária - CN\r\n\t\t\t\t\t\t\t&dtHoraQuarto=12:37\r\n\t\t\t\t\t\t\t&txEtapa=',
  'TextoHTML.asp?etapa=5\r\n\t\t\t\t\t\t\t&nuSessao=013

In [84]:
len(link_infos[0])

100

In [85]:
linkDiscurso = []
temp = []
for i in range(qpt):
    for item in link_infos[i]:
        text = item.replace('\t','').replace('\n','').replace('\r','').replace(' ','')
        temp.append(("https://www.camara.leg.br/internet/SitaqWeb/"+text))
    linkDiscurso.append(temp)
    temp = []

In [86]:
len(linkDiscurso[0])

100

In [87]:
len(linkDiscurso[0])

100

In [88]:
linkDiscurso

[['https://www.camara.leg.br/internet/SitaqWeb/TextoHTML.asp?etapa=5&nuSessao=013.4.56.N&nuQuarto=0&nuOrador=0&nuInsercao=3&dtHorarioQuarto=12:37&sgFaseSessao=OD&Data=28/04/2022&txApelido=AFONSOFLORENCE,PT-BA&txFaseSessao=OrdemdoDia&txTipoSessao=Extraordinária-CN&dtHoraQuarto=12:37&txEtapa=',
  'https://www.camara.leg.br/internet/SitaqWeb/TextoHTML.asp?etapa=5&nuSessao=013.4.56.N&nuQuarto=0&nuOrador=0&nuInsercao=23&dtHorarioQuarto=12:37&sgFaseSessao=OD&Data=28/04/2022&txApelido=AFONSOFLORENCE,PT-BA&txFaseSessao=OrdemdoDia&txTipoSessao=Extraordinária-CN&dtHoraQuarto=12:37&txEtapa=',
  'https://www.camara.leg.br/internet/SitaqWeb/TextoHTML.asp?etapa=5&nuSessao=013.4.56.N&nuQuarto=0&nuOrador=0&nuInsercao=27&dtHorarioQuarto=12:37&sgFaseSessao=OD&Data=28/04/2022&txApelido=AFONSOFLORENCE,PT-BA&txFaseSessao=OrdemdoDia&txTipoSessao=Extraordinária-CN&dtHoraQuarto=12:37&txEtapa=',
  'https://www.camara.leg.br/internet/SitaqWeb/TextoHTML.asp?etapa=5&nuSessao=013.4.56.N&nuQuarto=0&nuOrador=0&nuIns

In [89]:
txt = []
for script in linkDiscurso[0]:
    url_script = UrlToSoup(script).find_all("font")
    txt.append((str((url_script[0].contents)) + "+" + str((url_script[1].contents))).replace('<br/>','').replace('<b>', '').replace('</b>', '').replace('\'', ''))
txt


['[O SR. AFONSO FLORENCE ]+[(Bloco/PT - BA. Pela Liderança. Sem revisão do orador.) - Presidente Marcelo Ramos, quero trazer aqui a posição da Liderança da Minoria, com o mandato que exerço de Vice-Líder com a delegação do Líder Alencar., , Primeiro me permita fazer uma saudação à condução de V. Exa. como Presidente da sessão, como Vice-Presidente do Congresso Nacional. O Brasil sabe que V. Exa. foi eleito com voto dos Parlamentares para exercer um mandato de Vice-Presidente, e a condução, inúmeras vezes não correspondendo aos interesses políticos, regimentais, da Oposição - possivelmente, eventualmente, também não ao Governo -, pode ser caracterizada como reta, em respeito à Constituição, ao Regimento, e uma conduta imparcial, o que se repete nesse caso, nesse momento que é um momento nevrálgico, porque tem marcha dos Prefeitos, tem crise econômica, carestia e aumento da fome, mas, como V. Exa. diz, é escravo da Constituição e do Regimento. , , Então, antes de tentar contribuir com a 

In [90]:
for i in range(len(txt)):
    if not(os.path.exists(f"./discursos/file_{i}.txt")):
        textfile = open(f"./discursos/file_{i}.txt", "x")
        textfile.close()

In [91]:
for i in range(len(txt)):

    textfile = open(f"./discursos/file_{i}.txt", "w")
    textfile.write(txt[i] + "\n")
    textfile.close()

In [92]:
id_list = []
id = []
for i in range(len(linkDiscurso[0])):
    id.append('nI:')
    id.append((re.findall(r'(?<=nuInsercao=)\d+', linkDiscurso[0][i]))[0])
    id.append("nQ:")
    id.append((re.findall(r'(?<=nuQuarto=)\d+', linkDiscurso[0][i]))[0])
    id.append("nS:")
    id.append((re.findall(r'(?<=nuSessao=)\d+', linkDiscurso[0][i]))[0])
    id.append("nO")
    id.append((re.findall(r'(?<=nuOrador=)\d+', linkDiscurso[0][i]))[0])
    id_list.append("".join(id))
    id = []




In [93]:
id_list

['nI:3nQ:0nS:013nO0',
 'nI:23nQ:0nS:013nO0',
 'nI:27nQ:0nS:013nO0',
 'nI:33nQ:0nS:013nO0',
 'nI:37nQ:0nS:013nO0',
 'nI:40nQ:0nS:013nO0',
 'nI:41nQ:0nS:013nO0',
 'nI:46nQ:0nS:013nO0',
 'nI:48nQ:0nS:013nO0',
 'nI:52nQ:0nS:013nO0',
 'nI:6nQ:0nS:013nO0',
 'nI:1nQ:0nS:013nO0',
 'nI:2nQ:0nS:013nO0',
 'nI:5nQ:0nS:013nO0',
 'nI:14nQ:0nS:013nO0',
 'nI:22nQ:0nS:013nO0',
 'nI:10nQ:0nS:013nO0',
 'nI:18nQ:0nS:013nO0',
 'nI:35nQ:0nS:013nO0',
 'nI:21nQ:0nS:013nO0',
 'nI:42nQ:0nS:013nO0',
 'nI:51nQ:0nS:013nO0',
 'nI:44nQ:0nS:013nO0',
 'nI:60nQ:0nS:013nO0',
 'nI:17nQ:0nS:013nO0',
 'nI:16nQ:0nS:013nO0',
 'nI:31nQ:0nS:013nO0',
 'nI:56nQ:0nS:013nO0',
 'nI:8nQ:0nS:013nO0',
 'nI:19nQ:0nS:013nO0',
 'nI:12nQ:0nS:013nO0',
 'nI:25nQ:0nS:013nO0',
 'nI:4nQ:0nS:013nO0',
 'nI:28nQ:0nS:013nO0',
 'nI:29nQ:0nS:013nO0',
 'nI:7nQ:0nS:013nO0',
 'nI:11nQ:0nS:013nO0',
 'nI:15nQ:0nS:013nO0',
 'nI:34nQ:0nS:013nO0',
 'nI:45nQ:0nS:013nO0',
 'nI:47nQ:0nS:013nO0',
 'nI:50nQ:0nS:013nO0',
 'nI:43nQ:0nS:013nO0',
 'nI:39nQ:0nS:013nO

In [141]:
data_frame[0][0]['Id'] = id_list

In [143]:
data_frame[0][0]

,Data,Sessão,Fase,Discurso,Sumário,Orador,Hora,Publicação,Id
0,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022,nI:3nQ:0nS:013nO0
1,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022,nI:23nQ:0nS:013nO0
2,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022,nI:27nQ:0nS:013nO0
3,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022,nI:33nQ:0nS:013nO0
4,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022,nI:37nQ:0nS:013nO0
...,...,...,...,...,...,...,...,...,...
95,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"REINHOLD STEPHANES JUNIOR, PSD-PR",15h52,DCD 30/03/2022,nI:3nQ:553460nS:25nO3
96,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"FRANCO CARTAFINA, PP-MG",15h56,DCD 30/03/2022,nI:3nQ:553461nS:25nO3
97,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"MARCEL VAN HATTEM, NOVO-RS",16h00,DCD 30/03/2022,nI:3nQ:553462nS:25nO3
98,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"DANIEL SILVEIRA, UNIÃO-RJ",16h04,DCD 30/03/2022,nI:3nQ:553463nS:25nO3


In [146]:
for col in data_frame[0][0]:
    print(data_frame[0][0][col].unique())

['28/04/2022' '26/04/2022' '05/04/2022' '29/03/2022']
['013.4.56.N' '011.4.56.N' '009.4.56.N' '25.2022']
['ORDEM DO DIA' 'HOMENAGEM' 'BREVES COMUNICAÇÕES']
[nan]
[nan]
['AFONSO FLORENCE, PT-BA' 'AGUINALDO RIBEIRO, PP-PB'
 'ANTONIO BRITO, PSD-BA' 'ARLINDO CHINAGLIA, PT-SP'
 'BIRA DO PINDARÉ, PSB-MA' 'CARMEN ZANOTTO, CIDADANIA-SC'
 'CHRISTIANE DE SOUZA YARED, PL-PR' 'CLAUDIO CAJADO, PP-BA'
 'DR. ZACARIAS CALIL, DEM-GO' 'FÁBIO TRAD, PSD-MS' 'HEITOR SCHUCH, PSB-RS'
 'IVAN VALENTE, PSOL-SP' 'LEÔNIDAS CRISTINO, PDT-CE'
 'MÁRCIO MACÊDO, PT-SE' 'NELHO BEZERRA, UNIÃO-CE'
 'OTTACI NASCIMENTO, SOLIDARIEDADE-RR' 'PAULO TEIXEIRA, PT-SP'
 'RICARDO BARROS, PP-PR' 'SORAYA SANTOS, PL-RJ' 'TIAGO MITRAUD, NOVO-MG'
 'VALTENIR PEREIRA, MDB-MT' 'VINICIUS CARVALHO, REPUBLICANOS-SP'
 'VIVI REIS, PSOL-PA' 'WELITON PRADO, PROS-MG'
 'GENERAL PETERNELLI, UNIÃO-SP' 'MARGARETE COELHO, PP-PI'
 'FREI ANASTACIO RIBEIRO, PT-PB' 'VALMIR ASSUNÇÃO, PT-BA'
 'ROSANGELA GOMES, REPUBLICANOS-RJ' 'GENERAL GIRÃO, PL-RN'
 'HELDER

In [147]:
data_frame[0][0]['Id'].nunique()

100